In [ ]:
pip install git+https://github.com/UN-GCPDS/python-gcpds.utils.git

In [ ]:
!pip install mne==0.19

In [ ]:
FILEID = "1A2a60KUPxtUboqlb8B6Q0U-OygVmS_wK"
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id='$FILEID -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id="$FILEID -O CKA_keras.py && rm -rf /tmp/cookies.txt
!dir

In [ ]:
#%% download data
FILEID = "1DOm75xoSPUbnFfcJBjVggPS0hzNEoWCG"
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id='$FILEID -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id="$FILEID -O MIfunctions.py && rm -rf /tmp/cookies.txt
#%% download data

In [ ]:
FILEID = "1woC5hNp_n7U2Ye4KW1TZxOS17KdcpwV4"
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id='$FILEID -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id="$FILEID -O physionet.zip && rm -rf /tmp/cookies.txt
!dir
!unzip physionet.zip

In [ ]:
from gcpds.utils import loaddb
import sys
from scipy.io import loadmat
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
import numpy as np
from joblib import dump, load
from sklearn.svm import LinearSVC, SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import train_test_split,GridSearchCV, StratifiedShuffleSplit
from mne.viz import plot_topomap
#from keras_cka_v3 import Keras_CKA # import keras_cka function wrapped for sklearn.
from CKA_keras import Keras_CKA # import keras_cka function wrapped for sklearn.
from MIfunctions import * #importar todas las funciones de manejo de eeg con mne.
import os
import time
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, cohen_kappa_score, pairwise_distances
from sklearn.metrics import make_scorer

In [ ]:
from os import listdir
from os.path import isfile, isdir
path='/content/physionet'
listdir = listdir(path)

In [ ]:
sbj = listdir[0]
sbj

'S003'

In [ ]:
tau =  4
overlap=1-0.5
ti = 4
tf = 40
tti = np.arange(ti,tf-tau+(tau*overlap),tau*overlap)
ttf = np.arange(ti+tau,tf+tau*overlap,tau*overlap)
f_frec = np.array([tti,ttf]).T
#vtw= np.array([[0,2],[1,3],[2,4],[3,5],[4,6],[5,7]])
tau =  1.5
step = tau * (1-0.5)
ti = 2
tf = 6
tti = np.arange(ti,tf-tau+(step),step)
ttf = np.arange(ti+tau,tf+step,step)
vtw = np.array([tti,ttf]).T
db = loaddb.GIGA('Giga')
for j in range(len(listdir)):
    sbj = listdir[j]
    XX = loadmat('/content/physionet/'+sbj+'/eeg/raw.mat')
    data=XX['X']
    data[0][0].shape
    Xraw = np.array([data[0][j] for j in range(len(data[0]))])
    fs = XX['fs'][0][0]
    ch = XX['channels'][0]
    channels_names = [ch[c][0] for c in range(len(ch))]
    yy = XX['y']
    y = np.array([yy[j][0] for j in range(len(yy))])-1
    montage = mne.channels.read_montage(db.metadata['montage'], channels_names)
    info = mne.create_info(channels_names, sfreq=fs, ch_types="eeg",
                            montage=montage)
    #opt = tf.keras.optimizers.RMSprop(learning_rate=1e-1)
    steps=[('CSP_window_time',Window_band_CSP_eppoch(fs=fs,vtw=vtw,f_frec=f_frec)),
          ('flat',flatt()),
          ('stand',StandardScaler()),
          ('proy',Lasso_feats()),
           ('cla', LDA())]
    method = Pipeline(steps,memory='datospipeline')
    parameters ={'CSP_window_time__ncomp':[6],
            'proy__alpha':[1e-5,0.2,0.4,0.6,0.8,1]}
    scores = {'acc': 'accuracy','kappa': make_scorer(cohen_kappa_score,greater_is_better=True)}    
    cv = StratifiedShuffleSplit(n_splits=10, test_size=0.2, random_state=0)
    grid_search = GridSearchCV(method, parameters,cv=cv,verbose=10,
                               scoring=scores,refit='acc',n_jobs=-1)
    #grid_search = GridSearchCV(method, parameters,cv=cv,verbose=10,
     #                          scoring=kappa_,n_jobs=-1)
    #try:
    grid_search.fit(Xraw,y)
    results = grid_search.cv_results_
    filters = grid_search.best_estimator_['CSP_window_time'].filters
    ncomp = grid_search.best_estimator_['CSP_window_time'].ncomp
    #%% topoplots
    grid_search.best_estimator_.fit(Xraw,y)
    lasso=grid_search.best_estimator_['proy']
    rho = abs(lasso.lasso.coef_)
    rho = rho.reshape((len(vtw),ncomp,-1))
    rhocka = np.zeros((len(vtw),len(f_frec),len(channels_names)))
    ##
    srho = np.sum(rhocka,axis=2)
    r,c = srho.shape
    srho = (srho/np.sum(srho)).ravel()
    indx = np.argsort(srho)[::-1]
    ordsrho = srho[indx]
    cumsumrho = np.cumsum(ordsrho)
    xx = np.arange(len(cumsumrho))+1
    indd = np.argmin(np.sqrt((1-cumsumrho)**2+(xx/np.max(xx))**2))
    vfs = indx[0:xx[indd]]
    Wind_m = vfs//c
    Freq_m = vfs-(Wind_m*c)
    ##
    if np.logical_and(len(f_frec)>1,len(vtw)>1):
        for i in range(len(vtw)):
          for j in range(len(f_frec)):
            rhocka[i,j,:] =  np.sum(np.diag(np.abs(rho[i,:,j])).dot(np.abs(filters[i][:,:,j])),0)
        rhocka -= np.min(rhocka)
        rhocka /= np.max(rhocka)
        f,ax = plt.subplots(len(vtw),len(f_frec),figsize=(20,20))
        cmap='jet'
        for ii in range(len(vtw)):
          for jj in range(len(f_frec)):
              plot_topomap(rhocka[ii,jj,:], info, axes=ax[ii,jj], show=False,cmap=cmap,vmin=np.min(rhocka),vmax=np.max(rhocka))
              ax[ii,jj].set_title('Band '+str(f_frec[jj])+'[Hz]'+'ventana '+str(vtw[ii]))
        cax = f.add_axes([0.95, 0.15, 0.02, 0.75])
        sm = plt.cm.ScalarMappable(cmap=cmap)
        sm.set_array([])
        plt.colorbar(sm,cax=cax)
    elif np.logical_and(len(f_frec)>1,len(vtw)==1):
        for j in range(len(f_frec)):
          rhocka[0,j,:] =  np.sum(np.diag(np.abs(rho[0,:,j])).dot(np.abs(filters[0][:,:,j])),0)
        rhocka=rhocka.squeeze()
        rhocka -= np.min(rhocka)
        rhocka /= np.max(rhocka)
        f,ax = plt.subplots(len(f_frec),1,figsize=(20,20))
        cmap='jet'
        for jj in range(len(f_frec)):
            plot_topomap(rhocka[jj,:], info, axes=ax[jj], show=False,cmap=cmap,vmin=np.min(rhocka),vmax=np.max(rhocka))
            ax[jj].set_title('Band '+str(f_frec[jj])+'[Hz]'+'ventana '+str(vtw))
        cax = f.add_axes([0.95, 0.15, 0.02, 0.75])
        sm = plt.cm.ScalarMappable(cmap=cmap)
        sm.set_array([])
        plt.colorbar(sm,cax=cax)  
    else:
        for i in range(len(vtw)):
            rhocka[i,:] =  np.sum(np.diag(np.abs(rho[i,:,0])).dot(np.abs(filters[i][:,:,0])),0)
        rhocka -= np.min(rhocka)
        rhocka /= np.max(rhocka)
        f,ax = plt.subplots(len(vtw),1,figsize=(20,20))
        cmap='jet'
        for ii in range(len(vtw)):
              plot_topomap(rhocka[ii,0,:], info, axes=ax[ii], show=False,cmap=cmap,vmin=np.min(rhocka),vmax=np.max(rhocka))
              ax[ii].set_title('Band '+str(f_frec[0])+'[Hz]'+'ventana '+str(vtw[ii]))
        cax = f.add_axes([0.95, 0.15, 0.02, 0.75])
        sm = plt.cm.ScalarMappable(cmap=cmap)
        sm.set_array([])
        plt.colorbar(sm,cax=cax)   
    plt.savefig('sbj'+str(sbj)+'acc'+str(np.round(grid_search.best_score_,2))+'.eps',format='eps')#,dpi=300)
    #plt.savefig('sbj'+str(sbj)+'kappa'+str(np.round(grid_search.best_score_,2))+'.png',format='png')#,dpi=300)
    savedata ={
        'Wind_m':Wind_m,
        'Freq_m':Freq_m,
        'steps':steps,
        'f_frec':f_frec,
        'vtw':vtw,
        'best_param':grid_search.best_params_,
        'rhocka':rhocka,        
        'cv_results':results
    }
    #dump(savedata,'sujeto_'+str(sbj)+'_cka_featuresCSP_BCI2a_acc.joblib')
    dump(savedata,'sujeto_'+str(sbj)+'_lasso_featuresCSP_GIGA_acc.joblib')
    print('sbj'+str(sbj)+'acc'+str(np.round(grid_search.best_score_*100,4)))
    !zip Results-joblib.zip ./*.joblib
    !zip Results-images.zip ./*.eps

Output hidden; open in https://colab.research.google.com to view.